<a href="https://colab.research.google.com/github/vap326/cse337/blob/main/lab4_dyna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4: TD and Dyna




## Exercise 1: Implement SARSA with n-step TD (n=5) on CliffWalking

**Objective:**  
In this exercise, you will implement the **SARSA algorithm** using **n-step temporal-difference learning with n=5**. You will apply your implementation to the **CliffWalking environment** in Gymnasium, and analyze how multi-step returns influence learning compared to standard 1-step SARSA.

---

### Environment
- Use `CliffWalking-v1`

---

### Instructions
1. Implement **SARSA with n-step TD updates (n=5)**:
   - Maintain an action-value table \(Q(s,a)\).
   - Use ε-greedy exploration.
   - Store states, actions, and rewards for the last 5 steps.
   - After each step, compute the n-step return: G_t
   - Update \(Q(s_t,a_t)\) toward \(G_t\).

2. Train your agent for several thousand episodes (e.g., 5,000).

3. Plot the **episode rewards over time** to visualize learning progress.

4. Compare qualitatively with 1-step SARSA:
   - Does n-step SARSA converge faster or slower?
   - How do the policies differ near the cliff?

---

### Deliverables
- Python code implementing SARSA with TD(5) (notebook in Github).  
- A plot of episode number vs episode return (plot in a cell below).  
- A short discussion (1 paragraph) comparing the results with standard SARSA.  


In [6]:
"""
Starter code for Exercise (you can use this code, or extend your code from previous lab)
Implement SARSA with TD(5) on CliffWalking-v1
"""

import numpy as np
import gymnasium as gym
from collections import deque
import matplotlib.pyplot as plt

# Environment
env = gym.make("CliffWalking-v1")

# Parameters
n_states = env.observation_space.n
n_actions = env.action_space.n
alpha = 0.1           # step size (learning rate)
gamma = 0.99          # discount factor
epsilon = 0.1         # epsilon for epsilon-greedy policy
n_step = 5           # number of steps for TD(n)
n_episodes = 5000

#maintain a buffer for all the transitions
# Why buffer for TDs?
# → Maintaining a buffer of the last n states, actions, and rewards
#   allows us to compute n-step returns efficiently by looking back
#   at recent trajectories instead of waiting until the episode ends.

# Initialize Q-table
Q = np.zeros((n_states, n_actions))

def epsilon_greedy(state):
    """Choose an action using epsilon-greedy policy."""
    if np.random.rand() < epsilon:
        return np.random.randint(n_actions)
    return np.argmax(Q[state])

# Track returns
episode_returns = []

for ep in range(n_episodes):
    state, _ = env.reset()
    action = epsilon_greedy(state)

    # Buffers to store the trajectory
    states = deque()
    actions = deque()
    rewards = deque()

    T = float("inf")
    t = 0
    G = 0
    done = False

    while True:
        if t < T:
            # Take real step in the environment
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated

            states.append(state)
            actions.append(action)
            rewards.append(reward)

            if done:
                T = t + 1
            else:
                next_action = epsilon_greedy(next_state)
                state = next_state
                action = next_action

        # Time index for state/action to update
        tau = t - n_step + 1
        if tau >= 0:
            # TODO: Compute the n-step return G for state tau
            # Hint: use rewards[tau : tau+n] plus Q(s_t+n, a_t+n) if not terminal
            # Convert buffers to lists for safe indexing
            states_list = list(states)
            actions_list = list(actions)
            rewards_list = list(rewards)

            # Compute the n-step return
            G = 0.0
            for i in range(tau, min(tau + n_step, T)):
                G += (gamma ** (i - tau)) * rewards_list[i]

            if tau + n_step < T and (tau + n_step) < len(states_list):
                s_tau_n = states_list[tau + n_step]
                a_tau_n = actions_list[tau + n_step]
                G += (gamma ** n_step) * Q[s_tau_n, a_tau_n]

            # Update Q
            s_tau = states_list[tau]
            a_tau = actions_list[tau]
            Q[s_tau, a_tau] += alpha * (G - Q[s_tau, a_tau])

        if tau == T - 1:
            break

        t += 1

    episode_returns.append(sum(rewards))

# Plot learning curve
plt.plot(episode_returns)
plt.xlabel("Episode")
plt.ylabel("Return")
plt.title("SARSA with TD(5) on CliffWalking")
plt.show()


KeyboardInterrupt: 

## Exercise 2: Dyna-Q for CliffWalking

**Objective**  
Implement **Dyna-Q** on **CliffWalking-v1** and compare its learning performance to **SARSA (1-step)** and **SARSA TD(5)**. You will analyze sample efficiency, stability near the cliff, and sensitivity to planning steps.

---

### Environment
- Use `CliffWalking-v1`
---

### Part A — Dyna-Q (Implementation)
1. **Q-table**: maintain `Q[s, a]` (tabular).
2. **Model**: learn an empirical model from experience.
   - For each observed transition `(s, a, r, s')`, update a dictionary:
     - Minimal: store the most recent `(s', r)` for `(s, a)`, **or**
     - Advanced: store a **multiset** of outcomes for `(s, a)` with counts (to sample stochastically).
3. **Real update (Q-learning)** after each env step:
   Q(s,a) ← Q(s,a) + α * (r + γ * max_a' Q(s',a') - Q(s,a))
4. **Planning updates**: after each real step, perform `N` simulated updates:
   - Sample a previously seen `(s_p, a_p)` from the model.
   - Sample `(r_p, s'_p)` from that entry.
   - Apply the same Q-learning backup using `(s_p, a_p, r_p, s'_p)`.
5. Use epsilon-greedy exploration.

---

### Part B — Baselines (Re-use / Implement)
- **SARSA (1-step)** with ε-greedy:
  \[
  Q(s,a) \leftarrow Q(s,a) + \alpha\big[r + \gamma Q(s',a') - Q(s,a)\big]
  \]
- **SARSA TD(5)** (n-step SARSA with \(n=5\)), as in Exercise 1.

Use the **same** γ, α, ε schedule, and number of episodes for a fair comparison.

---

### Part C — Experiments & Comparisons
1. **Learning curves**: plot **episode index vs. episode return** for:
   - Dyna-Q with \(N \in \{5, 20, 50\}\)
   - SARSA (1-step)
   - SARSA TD(5)
2. **Sample efficiency**: report the **episode number** at which the average return over a sliding window (e.g., 100 episodes) first exceeds a chosen threshold (e.g., −30).
3. **Stability near the cliff**: qualitatively inspect trajectories/policies; does the method hug the cliff or leave a safer margin?
4. **Sensitivity to planning steps**: compare Dyna-Q across N; discuss diminishing returns vs. computation.
5. **Statistical robustness**: run **≥5 seeds**; plot mean ± std (shaded) or report mean ± std of final returns.

---

### Deliverables
- **Code**: A driver script/notebook that reproduces your plots
- **Plots** (embedded in the notebook):
  - Learning curves (mean ± std across seeds)
  - Optional: heatmap of greedy policy/actions on the grid




In [9]:
# Dyna-Q vs SARSA(1) vs SARSA TD(5) on CliffWalking-v1
# Run in a Jupyter cell. Requires: gymnasium, numpy, matplotlib

import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from collections import deque, defaultdict
import random
from copy import deepcopy

# -------- Hyperparameters and experiment config --------
gamma = 0.99
alpha = 0.1
epsilon = 0.1

n_episodes = 2000           # number of episodes per run (adjustable)
seeds = [0, 1, 2, 3, 4]     # run >=5 seeds for statistical robustness
window = 100                # sliding window for sample-efficiency threshold
threshold = -30             # example threshold for average return (CliffWalking returns are negative)

# Dyna planning steps to compare
dyna_N_list = [5, 20, 50]

# For reproducibility across gym resets we set seed per env when creating it in each run

# -------- Helper functions --------
def epsilon_greedy(Q, state, n_actions, eps):
    if np.random.rand() < eps:
        return np.random.randint(n_actions)
    return int(np.argmax(Q[state]))

def run_sarsa_1(env_name, seed, n_episodes, alpha, gamma, epsilon):
    env = gym.make(env_name)
    env.reset(seed=seed)
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    Q = np.zeros((n_states, n_actions))
    returns = []

    for ep in range(n_episodes):
        state, _ = env.reset(seed=(seed + ep))  # vary seed slightly per episode to vary randomness
        a = epsilon_greedy(Q, state, n_actions, epsilon)
        ep_return = 0
        done = False
        while True:
            next_state, reward, terminated, truncated, _ = env.step(a)
            done = terminated or truncated
            ep_return += reward

            if done:
                target = reward
                Q[state, a] += alpha * (target - Q[state, a])
                break
            else:
                next_a = epsilon_greedy(Q, next_state, n_actions, epsilon)
                target = reward + gamma * Q[next_state, next_a]
                Q[state, a] += alpha * (target - Q[state, a])

                state = next_state
                a = next_a

        returns.append(ep_return)

    env.close()
    return np.array(returns), Q

def run_sarsa_n(env_name, seed, n_episodes, alpha, gamma, epsilon, n_step=5):
    env = gym.make(env_name)
    env.reset(seed=seed)
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    Q = np.zeros((n_states, n_actions))
    returns = []

    for ep in range(n_episodes):
        state, _ = env.reset(seed=(seed + ep))
        action = epsilon_greedy(Q, state, n_actions, epsilon)

        states = deque()
        actions = deque()
        rewards = deque()

        T = float("inf")
        t = 0
        ep_return = 0

        while True:
            if t < T:
                next_state, reward, terminated, truncated, _ = env.step(action)
                done = terminated or truncated
                ep_return += reward

                states.append(state)
                actions.append(action)
                rewards.append(reward)

                if done:
                    T = t + 1
                else:
                    next_action = epsilon_greedy(Q, next_state, n_actions, epsilon)
                    state = next_state
                    action = next_action

            tau = t - n_step + 1
            if tau >= 0:
                # snapshot lists for safe indexing
                s_list = list(states)
                a_list = list(actions)
                r_list = list(rewards)

                # compute n-step return
                G = 0.0
                end = min(tau + n_step, T)   # ensure we don’t pass inf to range()
                for i in range(tau, min(tau + n_step, int(T))):
                    G += (gamma ** (i - tau)) * r_list[i]
                if tau + n_step < T:
                    # ensure index exists
                    idx = tau + n_step
                    if idx < len(s_list):
                        s_tau_n = s_list[idx]
                        a_tau_n = a_list[idx]
                        G += (gamma ** n_step) * Q[s_tau_n, a_tau_n]

                s_tau = s_list[tau]
                a_tau = a_list[tau]
                Q[s_tau, a_tau] += alpha * (G - Q[s_tau, a_tau])

                # pop left when we've updated the earliest stored element to avoid unbounded growth
                # but only pop if queue length > n_step to keep indexing consistent; we can pop once tau >= 0 and tau < len
                # A simpler safe approach: pop left now because we've already used s_tau (the earliest stored)
                states.popleft()
                actions.popleft()
                rewards.popleft()

            if tau == T - 1:
                break

            t += 1

        returns.append(ep_return)

    env.close()
    return np.array(returns), Q

def run_dyna_q(env_name, seed, n_episodes, alpha, gamma, epsilon, planning_steps=5, model_type="multi"):
    """
    model_type:
      - "one-step": model[(s,a)] = (r, s')  most recent (deterministic)
      - "multi": model[(s,a)] = list of (r, s') outcomes (sample uniformly)
    """
    env = gym.make(env_name)
    env.reset(seed=seed)
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    Q = np.zeros((n_states, n_actions))
    returns = []

    # model: dict from (s,a) -> list of (r,s')
    model = defaultdict(list)
    seen_sa = set()

    for ep in range(n_episodes):
        state, _ = env.reset(seed=(seed + ep))
        ep_return = 0
        done = False

        while True:
            a = epsilon_greedy(Q, state, n_actions, epsilon)
            next_state, reward, terminated, truncated, _ = env.step(a)
            done = terminated or truncated
            ep_return += reward

            # Q-learning real update
            if done:
                target = reward
            else:
                target = reward + gamma * np.max(Q[next_state])
            Q[state, a] += alpha * (target - Q[state, a])

            # update model
            if model_type == "one-step":
                model[(state, a)] = [(reward, next_state)]
            else:
                model[(state, a)].append((reward, next_state))
            seen_sa.add((state, a))

            # planning updates
            for _ in range(planning_steps):
                if len(seen_sa) == 0:
                    break
                s_p, a_p = random.choice(list(seen_sa))
                outcomes = model[(s_p, a_p)]
                r_p, s_next_p = random.choice(outcomes)
                # Q-learning backup on simulated experience
                if s_next_p is None:
                    target_p = r_p
                else:
                    target_p = r_p + gamma * np.max(Q[s_next_p])
                Q[s_p, a_p] += alpha * (target_p - Q[s_p, a_p])

            if done:
                break

            state = next_state

        returns.append(ep_return)

    env.close()
    return np.array(returns), Q

# -------- Run experiments (multiple seeds) --------
env_name = "CliffWalking-v1"

# containers to hold all runs
results = {}   # key -> list of returns arrays (one per seed)
policies = {}  # store final Q for last seed for quick policy inspection (optional)

# Baseline SARSA(1)
print("Running SARSA(1) across seeds...")
results['SARSA_1'] = []
for seed in seeds:
    r, Qfinal = run_sarsa_1(env_name, seed, n_episodes, alpha, gamma, epsilon)
    results['SARSA_1'].append(r)
    policies.setdefault('SARSA_1', []).append(Qfinal)

# SARSA TD(5)
print("Running SARSA TD(5) across seeds...")
results['SARSA_TD5'] = []
for seed in seeds:
    r, Qfinal = run_sarsa_n(env_name, seed, n_episodes, alpha, gamma, epsilon, n_step=5)
    results['SARSA_TD5'].append(r)
    policies.setdefault('SARSA_TD5', []).append(Qfinal)

# Dyna-Q with various planning steps
for N in dyna_N_list:
    key = f"DynaQ_N{N}"
    print(f"Running {key} across seeds...")
    results[key] = []
    for seed in seeds:
        r, Qfinal = run_dyna_q(env_name, seed, n_episodes, alpha, gamma, epsilon, planning_steps=N, model_type="multi")
        results[key].append(r)
        policies.setdefault(key, []).append(Qfinal)

# -------- Aggregate statistics across seeds --------
def aggregate_returns(list_of_arrays):
    # list_of_arrays: list of (n_episodes,) arrays
    arr = np.vstack(list_of_arrays)  # shape (n_seeds, n_episodes)
    mean = arr.mean(axis=0)
    std = arr.std(axis=0)
    return mean, std

aggregated = {}
for k, runs in results.items():
    mean, std = aggregate_returns(runs)
    aggregated[k] = (mean, std)

# -------- Sample efficiency: find first episode where sliding average (window) > threshold --------
def first_episode_exceeding_threshold(runs, window, threshold):
    # runs: list of arrays (each array length n_episodes)
    first_eps = []
    for run in runs:
        # compute sliding averages
        if len(run) < window:
            first_eps.append(None)
            continue
        movavg = np.convolve(run, np.ones(window)/window, mode='valid')  # length n_episodes - window +1
        idxs = np.where(movavg > threshold)[0]
        if len(idxs) == 0:
            first_eps.append(None)
        else:
            # convert index in movavg to episode number (we choose the *first* episode of the window that meets criterion)
            first_eps.append(int(idxs[0] + window - 1))
    # return mean ± std excluding None
    val_list = [v for v in first_eps if v is not None]
    if len(val_list) == 0:
        return None, None, first_eps
    return np.mean(val_list), np.std(val_list), first_eps

sample_efficiency = {}
for k, runs in results.items():
    mean_ep, std_ep, all_first = first_episode_exceeding_threshold(runs, window, threshold)
    sample_efficiency[k] = {'mean_first_ep': mean_ep, 'std_first_ep': std_ep, 'all_first': all_first}

# -------- Plot learning curves (mean ± std) --------
plt.figure(figsize=(12, 8))
for k, (mean, std) in aggregated.items():
    episodes = np.arange(1, len(mean)+1)
    plt.plot(episodes, mean, label=k)
    plt.fill_between(episodes, mean-std, mean+std, alpha=0.2)

plt.xlabel("Episode")
plt.ylabel("Return (episode total reward)")
plt.title("Learning curves: Dyna-Q (various N) vs SARSA(1) vs SARSA TD(5)\nMean ± Std over seeds")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# -------- Print sample-efficiency results --------
print("\nSample efficiency (first episode where sliding-window avg > {} over window={}):".format(threshold, window))
for k, v in sample_efficiency.items():
    mean_ep = v['mean_first_ep']
    std_ep = v['std_first_ep']
    if mean_ep is None:
        print(f"  {k}: never exceeded threshold in any seed. per-seed: {v['all_first']}")
    else:
        print(f"  {k}: first episode ≈ {mean_ep:.1f} ± {std_ep:.1f} (per-seed: {v['all_first']})")

# -------- Optional: visualize greedy policy heatmap for a method (uses last Q of last seed) --------
# CliffWalking is 4 x 12 grid (rows x cols), states numbered row-major
def plot_policy_arrows(Q, title="Greedy policy (arrows)"):
    n_states, n_actions = Q.shape
    n_rows, n_cols = 4, 12
    assert n_states == n_rows * n_cols
    # action mapping (CliffWalking): 0=UP,1=RIGHT,2=DOWN,3=LEFT
    action_symbols = {0: '^', 1: '>', 2: 'v', 3: '<'}
    policy = np.argmax(Q, axis=1)
    fig, ax = plt.subplots(figsize=(12,4))
    ax.set_xlim(0, n_cols)
    ax.set_ylim(0, n_rows)
    ax.invert_yaxis()
    ax.set_xticks(np.arange(0,n_cols))
    ax.set_yticks(np.arange(0,n_rows))
    ax.grid(True)
    for s in range(n_states):
        r = s // n_cols
        c = s % n_cols
        a = int(policy[s])
        ax.text(c+0.5, r+0.5, action_symbols[a], ha='center', va='center', fontsize=14)
    ax.set_title(title)
    plt.show()

# Pick final Q from each method's last seed and plot
for k, qlist in policies.items():
    Q_last = qlist[-1]  # last seed's final Q
    plot_policy_arrows(Q_last, title=f"Greedy policy — {k}")

# -------- End of script --------


Running SARSA(1) across seeds...
Running SARSA TD(5) across seeds...


OverflowError: cannot convert float infinity to integer

## Exercise 3: Solve FrozenLake with Q-Learning and Dyna-Q (Stochastic Model)

**Objective**  
Implement and compare **Q-learning** and **Dyna-Q** on Gymnasium’s `FrozenLake-v1`.  
For Dyna-Q, your learned **transition model must handle multiple possible next states** per `(s, a)` (stochastic slip), i.e., store and sample **a distribution** over `(s', r)` outcomes rather than a single next state.

---

### Environment
- Use `FrozenLake-v1` from `gymnasium.envs.toy_text`.
- You can start with map 4×4; and then work with 8×8.
- Start → Goal with slippery transitions (stochastic).  
- Rewards: `+1` at goal, `0` otherwise (holes terminate with 0).

---

### Part A — Q-learning (baseline)
1. Maintain a tabular action-value function `Q[s, a]`.
2. Behavior: ε-greedy over `Q`.
3. Update after each real step:
   - target = r + γ * max_a' Q[s', a']   (if terminal: target = r)
   - Q[s, a] ← Q[s, a] + α * (target − Q[s, a])
4. Train for several thousand episodes (e.g., 5,000) with an ε schedule (e.g., 0.2 → 0.01).

---

### Part B — Dyna-Q with a **stochastic transition model**
1. **Empirical model (multinomial):** for each `(s, a)`, maintain a multiset of observed outcomes:
   - `model[(s, a)] = [(s'_1, r_1, count_1), (s'_2, r_2, count_2), ...]`
   - Update counts whenever you observe `(s, a, r, s')`.
2. **Real step update (Q-learning):** same as Part A.
3. **Planning steps (N per real step):**
   - Sample a previously seen `(s_p, a_p)` uniformly (or with priority).
   - Sample `(s'_p, r_p)` **from the empirical distribution** for `(s_p, a_p)` using counts as probabilities.
   - Apply the same Q-learning backup with `(s_p, a_p, r_p, s'_p)`.
4. Train with the same ε schedule and number of episodes; vary `N ∈ {5, 20, 50}`.

---

### Experiments & Analysis
1. **Learning curves:** plot episode index vs episode return (smoothed) for:
   - Q-learning
   - Dyna-Q (N=5, 20, 50)
2. **Sample efficiency:** report the episode at which the moving-average return (e.g., window 100) first exceeds a threshold (you choose a reasonable value).
3. **Effect of stochastic modeling:** briefly explain why storing a distribution over `(s', r)` matters on FrozenLake (slip), and what happens if you store only the most recent outcome.
4. **Robustness:** run ≥5 random seeds; report mean ± std of final evaluation returns.

---

### Deliverables
- **Code** for Q-learning and Dyna-Q (with stochastic model).  
- **Plots** of learning curves (include legend and axis labels).  
- ** Discussion:** why Dyna-Q helps here; impact of N; importance of modeling multiple next states.

---

### Hints
- For terminal transitions (goal/hole), the Q-learning target is simply `target = r` (no bootstrap).  
- When sampling from the model, use probabilities `p_i = count_i / sum_j count_j`.  
- Tie-break greedy action selection uniformly among argmax actions to avoid bias.  
- Keep evaluation **greedy (ε=0)** and consistent across methods (same seeds and episode counts).


In [10]:
# FrozenLake: Q-Learning vs Dyna-Q (stochastic model)
# Requires: gymnasium, numpy, matplotlib
# Place in a Jupyter cell and run.

import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import random

# -------------------------
# Experiment configuration
# -------------------------
map_name = "4x4"              # "4x4" or "8x8"
is_slippery = True            # stochastic transitions (slip)
env_name = "FrozenLake-v1"

gamma = 0.99
alpha = 0.1
eps_start = 0.2
eps_end = 0.01
n_episodes = 5000            # per seed
seeds = [0,1,2,3,4]          # >=5 seeds for robustness
window = 100                 # moving average window for sample-efficiency
# thresholds: choose reasonable based on map size
threshold_by_map = {"4x4": 0.78, "8x8": 0.4}
threshold = threshold_by_map.get(map_name, 0.5)

dyna_N_list = [5, 20, 50]

# -------------------------
# Helpers
# -------------------------
def make_env(seed=None):
    # gymnasium FrozenLake-v1
    return gym.make(env_name, map_name=map_name, is_slippery=is_slippery)

def tie_break_argmax(q_values):
    """Return an argmax index breaking ties uniformly."""
    top = np.flatnonzero(q_values == np.max(q_values))
    return int(np.random.choice(top))

def epsilon_by_episode(ep, total_eps=n_episodes, start=eps_start, end=eps_end):
    """Linear decay from start to end across episodes."""
    frac = min(1.0, ep / float(total_eps))
    return start + frac * (end - start)

# -------------------------
# Q-Learning implementation
# -------------------------
def run_q_learning(seed, n_episodes=n_episodes):
    env = make_env(seed)
    env.reset(seed=seed)
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    Q = np.zeros((n_states, n_actions))
    returns = np.zeros(n_episodes)
    for ep in range(n_episodes):
        eps = epsilon_by_episode(ep)
        obs, _ = env.reset(seed=(seed + ep))
        done = False
        ep_ret = 0.0
        while True:
            # epsilon-greedy with tie-breaking
            if np.random.rand() < eps:
                a = np.random.randint(n_actions)
            else:
                a = tie_break_argmax(Q[obs])
            next_obs, reward, terminated, truncated, _ = env.step(a)
            done = terminated or truncated
            ep_ret += reward

            if done:
                target = reward
            else:
                target = reward + gamma * np.max(Q[next_obs])
            Q[obs, a] += alpha * (target - Q[obs, a])

            obs = next_obs
            if done:
                break
        returns[ep] = ep_ret
    env.close()
    return returns, Q

# -------------------------
# Dyna-Q with stochastic multinomial model
# -------------------------
class StochasticModel:
    """Empirical model storing counts of observed (s' , r) outcomes per (s,a)."""
    def __init__(self):
        # model[(s,a)] -> dict mapping s' -> count (rewards deterministic for FrozenLake: 1 at goal else 0)
        self.counts = defaultdict(lambda: defaultdict(int))
        self.rewards = {}  # optional map from s' to r - reward depends only on s' in FrozenLake
        self.sa_seen = set()

    def update(self, s, a, s_next, r):
        self.counts[(s,a)][s_next] += 1
        self.rewards[s_next] = r
        self.sa_seen.add((s,a))

    def sample(self, s, a):
        """Sample (s', r) according to observed empirical distribution for (s,a)."""
        outcomes = self.counts.get((s,a), None)
        if outcomes is None or len(outcomes) == 0:
            # unseen (s,a) -> no-op: return None
            return None
        states = list(outcomes.keys())
        counts = np.array([outcomes[s2] for s2 in states], dtype=float)
        probs = counts / counts.sum()
        s_next = np.random.choice(states, p=probs)
        r = self.rewards[s_next]
        return s_next, r

    def get_seen_pairs(self):
        return list(self.sa_seen)

def run_dyna_q(seed, planning_steps=5, n_episodes=n_episodes):
    env = make_env(seed)
    env.reset(seed=seed)
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    Q = np.zeros((n_states, n_actions))
    model = StochasticModel()
    returns = np.zeros(n_episodes)

    for ep in range(n_episodes):
        eps = epsilon_by_episode(ep)
        obs, _ = env.reset(seed=(seed + ep))
        ep_ret = 0.0
        while True:
            # epsilon-greedy policy with tie-breaking
            if np.random.rand() < eps:
                a = np.random.randint(n_actions)
            else:
                a = tie_break_argmax(Q[obs])

            next_obs, reward, terminated, truncated, _ = env.step(a)
            done = terminated or truncated
            ep_ret += reward

            # real Q-learning update
            if done:
                target = reward
            else:
                target = reward + gamma * np.max(Q[next_obs])
            Q[obs, a] += alpha * (target - Q[obs, a])

            # update model (store observed s' counts)
            model.update(obs, a, next_obs, reward)

            # planning: sample prior (s_p,a_p), then sample (s'_p,r_p) from model
            seen_pairs = model.get_seen_pairs()
            for _ in range(planning_steps):
                if len(seen_pairs) == 0:
                    break
                s_p, a_p = random.choice(seen_pairs)
                sampled = model.sample(s_p, a_p)
                if sampled is None:
                    continue
                s_next_p, r_p = sampled
                # simulated Q-learning update
                # if s_next_p terminal: reward is r_p and no bootstrap; handled since reward at terminal is r_p
                target_p = r_p + gamma * np.max(Q[s_next_p])
                Q[s_p, a_p] += alpha * (target_p - Q[s_p, a_p])

            obs = next_obs
            if done:
                break
        returns[ep] = ep_ret
    env.close()
    return returns, Q

# -------------------------
# Utilities: aggregate, moving average, sample-efficiency
# -------------------------
def aggregate_runs(runs):
    arr = np.vstack(runs)  # shape (n_seeds, n_episodes)
    mean = arr.mean(axis=0)
    std = arr.std(axis=0)
    return mean, std

def moving_average(x, w=window):
    if len(x) < w:
        return np.array([])
    return np.convolve(x, np.ones(w)/w, mode='valid')

def first_episode_exceeding(runs, w, threshold):
    # runs: list of 1d arrays
    firsts = []
    for r in runs:
        ma = moving_average(r, w)
        idxs = np.where(ma >= threshold)[0]
        if len(idxs) == 0:
            firsts.append(None)
        else:
            # convert moving-average index to episode index (use the last episode of the window)
            firsts.append(int(idxs[0] + w - 1))
    valid = [x for x in firsts if x is not None]
    if len(valid) == 0:
        return None, None, firsts
    return np.mean(valid), np.std(valid), firsts

# -------------------------
# Run experiments
# -------------------------
methods = {}
# Q-learning across seeds
print("Running Q-learning...")
ql_runs = []
ql_Qs = []
for seed in seeds:
    r, Qfinal = run_q_learning(seed)
    ql_runs.append(r)
    ql_Qs.append(Qfinal)
methods['Q-Learning'] = (ql_runs, ql_Qs)

# Dyna-Q with different N
for N in dyna_N_list:
    key = f"DynaQ_N{N}"
    print(f"Running {key} ...")
    runs = []
    Qs = []
    for seed in seeds:
        r, Qfinal = run_dyna_q(seed, planning_steps=N)
        runs.append(r)
        Qs.append(Qfinal)
    methods[key] = (runs, Qs)

# -------------------------
# Aggregate & Plot
# -------------------------
plt.figure(figsize=(12, 8))
aggregated = {}
for k, (runs, Qs) in methods.items():
    mean, std = aggregate_runs(runs)
    aggregated[k] = (mean, std)
    episodes = np.arange(1, len(mean)+1)
    plt.plot(episodes, moving_average(mean, w=1), label=k)  # raw mean (thin)
    # plot smoothed mean (window)
    sm = moving_average(mean, w=window)
    ep_sm = np.arange(window, len(mean)+1)
    plt.plot(ep_sm, sm, linewidth=2)
    plt.fill_between(ep_sm, sm - moving_average(std, w=window), sm + moving_average(std, w=window), alpha=0.2)

plt.xlabel("Episode")
plt.ylabel("Episode Return (0 or 1)")
plt.title(f"Q-Learning vs Dyna-Q (stochastic model) on FrozenLake {map_name} (slippery={is_slippery})\nSmoothed mean ± std (window={window})")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# -------------------------
# Sample efficiency reporting
# -------------------------
print("\nSample-efficiency (first episode when moving-average (w={}) >= {:.3f}):".format(window, threshold))
for k, (runs, Qs) in methods.items():
    mean_ep, std_ep, per_seed = first_episode_exceeding(runs, window, threshold)
    if mean_ep is None:
        print(f"  {k}: never exceeded threshold. per-seed: {per_seed}")
    else:
        print(f"  {k}: first episode ≈ {mean_ep:.1f} ± {std_ep:.1f}  (per-seed: {per_seed})")

# -------------------------
# Final greedy evaluation (ε=0) after training: run 100 evaluation episodes per seed
# -------------------------
def evaluate_greedy(Q, seed, n_eval=100):
    env = make_env(seed)
    env.reset(seed=seed)
    total = 0.0
    for ep in range(n_eval):
        obs, _ = env.reset(seed=(seed+ep))
        done = False
        while True:
            a = tie_break_argmax(Q[obs])
            obs, r, terminated, truncated, _ = env.step(a)
            done = terminated or truncated
            if done:
                total += r
                break
    env.close()
    return total / n_eval

print("\nFinal greedy evaluation (mean success rate over seeds):")
for k, (runs, Qs) in methods.items():
    evals = []
    for i, Q in enumerate(Qs):
        evals.append(evaluate_greedy(Q, seed=seeds[i], n_eval=200))
    print(f"  {k}: mean={np.mean(evals):.3f} ± std={np.std(evals):.3f} (per-seed evals: {np.round(evals,3)})")


Running Q-learning...
Running DynaQ_N5 ...


KeyboardInterrupt: 